In [1]:
import pandas as pd
import os
import numpy as np

In [2]:
smiles_df = pd.read_csv('Polymer-SMILES-FP.csv').drop_duplicates('SMILES')
smiles_df.head(10)

,index,SMILES,Polymer,__fingerprint_success__,afp_C2_C4_C4,afp_C2_C4_H1,afp_C3_C3_C3,afp_C3_C3_C4,afp_C3_C3_H1,afp_C3_C3_O1,...,mfp_MQNs36,mfp_MQNs37,mfp_MQNs38,mfp_MQNs39,mfp_MQNs40,mfp_MQNs41,mfp_MQNs42,mfp_NumAliphaticRings,mfp_NumAromaticRings,mfp_tpsa
0,0,[*]CC([*])C,poly(prop-1-ene),1,0.0,0.0,0.000000,0.000000,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
1,1,[*]CC([*])(C)C,polyisobutylene,1,0.0,0.0,0.000000,0.000000,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
2,2,[*]C[*],Linear HDPE,1,0.0,0.0,0.000000,0.000000,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
3,3,[*]CC([*])c1ccccc1,polystyrene,1,0.0,0.0,0.375000,0.125000,0.625000,0.0,...,0.062500,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.062500,0.000000
4,4,[*]CC([*])CCC,poly(pent-1-ene),1,0.0,0.0,0.000000,0.000000,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
6,6,[*]CC([*])(C)C(=O)OC,poly(methyl methacrylate),1,0.0,0.0,0.000000,0.000000,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.438333
7,7,[*]CC([*])(F)F,poly(vinylidene fluoride),1,0.0,0.0,0.000000,0.000000,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
9,9,[*]C[Si]([*])(CCCC)CCCC,Poly(ethyl-n-butylsilylene),1,0.0,0.0,0.000000,0.000000,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
10,10,[*]C[Si]([*])(C)C,Polycarbosilane,1,0.0,0.0,0.000000,0.000000,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
11,11,[*]CC([*])c1ccc(O)cc1,poly(p-hydroxystyrene),1,0.0,0.0,0.352941,0.117647,0.470588,0.0,...,0.058824,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.058824,0.297500


In [3]:
data = pd.read_excel('Melt Viscosity Data.xlsx', ["Homopolymer", "Copolymer"])
data["Copolymer"] = data["Copolymer"].drop(columns = ["INDEX", "PID"])
#data['Copolymer'].loc[0, data["Copolymer"].columns.isin(fp_columns)]
len(data["Homopolymer"])

691

In [4]:
fp_columns = smiles_df.columns[4:-1]
for i in data["Copolymer"].index:
    s_1 = data["Copolymer"].loc[i,"SMILES 1"]
    s_2 = data["Copolymer"].loc[i,"SMILES 2"]
    w_1 = data["Copolymer"].loc[i,"Weight 1"]
    w_2 = data["Copolymer"].loc[i,"Weight 2"]
    fp_1 = smiles_df.loc[smiles_df["SMILES"] == s_1, list(fp_columns.values)]
    fp_2 = smiles_df.loc[smiles_df["SMILES"] == s_2, list(fp_columns.values)]
    fp = (fp_1*w_1).add(fp_2*w_2, fill_value = 0).sum()
    data["Copolymer"].loc[i, fp_columns] = fp


In [5]:
data["Copolymer"]["Polymer 1"] = (data["Copolymer"]["Polymer 1"]  + ", " + data["Copolymer"]["Polymer 2"])
data["Copolymer"]["SMILES 1"] = (data["Copolymer"]["SMILES 1"] + ", " + data["Copolymer"]["SMILES 2"])
data["Copolymer"] = data["Copolymer"].rename(columns = {"Polymer 1": "Polymer", "SMILES 1": "SMILES"}).drop(columns = ["Polymer 2","SMILES 2"]).reset_index(drop=True)
data["Copolymer"]

In [6]:
data["Homopolymer"] = data["Homopolymer"].merge(smiles_df, how = 'left',on = 'SMILES') .rename(columns = {"Polymer_x": "Polymer"}).drop(columns = ["INDEX", "PID"])

In [47]:
out = pd.concat([data["Homopolymer"], data["Copolymer"]], ignore_index = True)
out = out.rename(columns = {"Melt Viscosity (poise)":"Melt_Viscosity", "Shear Rate":"Shear_Rate"}).rename_axis("INDEX")
#out.loc[,"Mw"] = 

In [56]:
#out["Mw"] = out["Mw"].fillna(np.nan)
#print(out["Mw"].isnull())
#out.loc[out["Mw"].isnull()] =
out.loc[out["Mw"].isnull() & out["Mn"].notnull(), "Mw"] = out.loc[out["Mw"].isnull() & out["Mn"].notnull() , "Mn"]*2

In [59]:
out.loc[289]

SAMPLE_ID                              NaN
SMILES                   [*]C[Si]([*])(C)C
Polymer                    Polycarbosilane
Mn                                  1090.0
Mw                                  2180.0
                               ...        
mfp_NumAliphaticRings                  0.0
mfp_NumAromaticRings                   0.0
mfp_tpsa                               0.0
Weight 1                               NaN
Weight 2                               NaN
Name: 289, Length: 194, dtype: object

In [60]:
out.to_excel('Data/full_data_10_08.xlsx')

In [62]:
zero_shear_data = out.loc[out["Shear_Rate"] == 0,:]
shear_data = out.loc[out["Shear_Rate"] != 0,:]

In [63]:
zero_shear_data.to_excel('Data/zero_shear_data_10_08.xlsx')
shear_data.to_excel('Data/shear_data_10_08.xlsx')